In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy 
from astropy.io import fits
from astropy.table import Table, Column, vstack, MaskedColumn, QTable
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky
from astropy.io import ascii
import os
import seaborn as sns
print(os.environ['PATH'])
os.environ['PATH'] = '/Library/TeX/texbin:' + os.environ['PATH']
print(os.environ['PATH'])
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 11
color_palette = sns.color_palette(["#000000", "#849324", "#FFB30F", "#D62828", "#6EA2D2"])

/Users/cassiemetzger/opt/anaconda3/bin:/Library/TeX/texbin:/Users/cassiemetzger/.gem/ruby/3.1.3/bin:/Users/cassiemetzger/.rubies/ruby-3.1.3/lib/ruby/gems/3.1.0/bin:/Users/cassiemetzger/.rubies/ruby-3.1.3/bin:/Users/cassiemetzger/opt/anaconda3/bin:/Users/cassiemetzger/opt/anaconda3/condabin:/opt/local/bin:/opt/local/sbin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/opt/X11/bin:/Library/Apple/usr/bin
/Library/TeX/texbin:/Users/cassiemetzger/opt/anaconda3/bin:/Library/TeX/texbin:/Users/cassiemetzger/.gem/ruby/3.1.3/bin:/Users/cassiemetzger/.rubies/ruby-3.1.3/lib/ruby/gems/3.1.0/bin:/Users/cassiemetzger/.rubies/ruby-3.1.3/bin:/Users/cassiemetzger/opt/anaconda3/bin:/Users/cassiemetzger/opt/anaconda3/condabin:/opt/local/bin:/opt/local/sbin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/opt/X11/bin:/Library/Apple/usr/bin


In [3]:
#input ALLWISE data (Cutri et al. 2013) AFTER it has been fed through wise_filter.sh 
hdu_list = fits.open('./catalogs/IR/wise.fits')
wise = Table(hdu_list[1].data)
hdu_list.close()

In [4]:
#input eRASS1_Main data (Merloni et al. 2024)
hdu_list = fits.open('./catalogs/xray/eRASS1_Main.v1.1.fits')
erass1 = Table(hdu_list[1].data)
hdu_list.close()

In [5]:
c = SkyCoord(ra = wise['ra']*u.degree, dec = wise['dec']*u.degree, frame = 'icrs')
catalog = SkyCoord(ra = erass1['RA']*u.degree, dec = erass1['DEC']*u.degree, frame = 'icrs')
idx, d2d, d3d = match_coordinates_sky(c, catalog)
m = d2d.arcsec <= 5 
wise['erass idx'] = idx 
matches = wise[m]

In [6]:
#assumes RA, DEC coords are given in degrees and radius is given in arcseconds
def matching(catalog1, RA1, DEC1, catalog2, RA2, DEC2, radius, matching_Name): 
    c = SkyCoord(ra = RA1*u.degree, dec = DEC1*u.degree, frame = 'fk5')
    catalog = SkyCoord(ra = RA2*u.degree, dec = DEC2*u.degree, frame = 'fk5')
    idx, d2d, d3d = c.match_to_catalog_sky(catalog)
    mask = d2d.arcsec <= radius
    matches = catalog1[mask]
    matches[f'{matching_Name} idx'] = idx[mask]
    matches[f'{matching_Name} sep'] = d2d.arcsec[mask]
    leng = len(matches)
    print(f'{leng} {matching_Name} matches found')
    print('Checking for duplicates...')
    numbers= matches[f'{matching_Name} idx']
    unique, counts = np.unique(numbers, return_counts=True)
    duplicates = unique[counts > 1] 
    print(f"Duplicates found: {len(duplicates)}")
    if(len(duplicates > 0)): 
        print(f"Removing duplicates...")
        mask = np.array([False if i in duplicates else True for i in matches[f'{matching_Name} idx']])
        for val in duplicates: 
            idx = np.where(matches[f'{matching_Name} idx'] == val)[0]
            dis1 = matches[f'{matching_Name} sep'][idx[0]]
            dis2 = matches[f'{matching_Name} sep'][idx[1]]
            if(dis1 < dis2): 
                mask[idx[0]] = True
                mask[idx[1]] = False
            if(dis2 < dis1): 
                mask[idx[0]] = False
                mask[idx[1]] = True
            if(dis1 == dis2): 
                mask[idx[0]] = False
                mask[idx[1]] = True
        matches = matches[mask]
        print(f"Removed {leng- len(matches)} duplicates \n {len(matches)} {matching_Name} matches found")
    return matches

In [7]:
matches = matching(wise, wise['ra'], wise['dec'], erass1, erass1['RA'], erass1['DEC'], 5, 'WISE')

5634 WISE matches found
Checking for duplicates...
Duplicates found: 152
Removing duplicates...
Removed 152 duplicates 
 5482 WISE matches found


In [8]:
def get_wise_flux(catalog): 
    sol = 2.997925E14 # variable sol; c in microns/s
    w1=sol/3.368
    w2=sol/4.618
    w3=sol/12.082

    fc_w1=0.9921
    fc_w2=0.9943
    fc_w3=0.9373

    F_w1 = 306.682 #Jy
    F_w2 = 170.663 #Jy
    F_w3 = 29.045 #Jy

    flux_w1 = F_w1/fc_w1*10**(-catalog['w1mpro']/2.5)
    flux_w2 = F_w2/fc_w2*10**(-catalog['w2mpro']/2.5)
    flux_w3 = F_w3/fc_w3*10**(-catalog['w3mpro']/2.5)

    #errors 
    w1errh = catalog['w1mpro'] - catalog['w1sigmpro']
    w2errh = catalog['w2mpro'] - catalog['w2sigmpro']
    w3errh = catalog['w3mpro'] - catalog['w3sigmpro']

    w1errl = catalog['w1mpro'] + catalog['w1sigmpro']
    w2errl = catalog['w2mpro'] + catalog['w2sigmpro']
    w3errl = catalog['w3mpro'] + catalog['w3sigmpro']

    #relative errors in Jy 

    err_w1_h = F_w1/fc_w1*10**(-w1errh/2.5)-flux_w1
    err_w2_h = F_w2/fc_w2*10**(-w2errh/2.5)-flux_w2
    err_w3_h = F_w3/fc_w3*10**(-w3errh/2.5)-flux_w3

    err_w1_l = F_w1/fc_w1*10**(-w1errl/2.5)-flux_w1
    err_w2_l = F_w2/fc_w2*10**(-w2errl/2.5)-flux_w2
    err_w3_l = F_w3/fc_w3*10**(-w3errl/2.5)-flux_w3

    err_w1 = 0.5*(np.abs(err_w1_h)+ np.abs(err_w1_l))
    err_w2 = 0.5*(np.abs(err_w2_h)+np.abs(err_w2_l))
    err_w3 = 0.5*(np.abs(err_w3_h)+np.abs(err_w3_l))

    flux = [flux_w1,flux_w2,flux_w3]
    err = [err_w1, err_w2, err_w3]
    catalog['w1flux'] = flux_w1 
    catalog['w2flux'] = flux_w2 
    catalog['w3flux'] = flux_w3
    catalog['w1sigflux'] = err_w1
    catalog['w2sigflux'] = err_w2
    catalog['w3sigflux'] = err_w3
    return catalog


In [9]:
matches = get_wise_flux(matches)

In [10]:
matches['ML_FLUX_1'] = erass1['ML_FLUX_1'][matches['erass idx']]

In [11]:
#choosing erosita 0.2-2.3 keV energy band, Eband = 1 keV 
e_energy = 1.6022e-16 #J 
h = 6.626e-34 #Js
e_v1 = e_energy/h
w_v = 3E8/3.4E-6


In [12]:
matches['ML_FLUX_1_JY'] = 1E+23 * matches['ML_FLUX_1'] * 1/(e_v1)
matches['Alpha IRX'] = -np.log10(matches['w1flux']/matches['ML_FLUX_1_JY'])/np.log10(w_v/e_v1)

In [13]:
matches['EXT_LIKE'] = erass1['EXT_LIKE'][matches['erass idx']]

In [14]:
x = matches['w2mpro'] - matches['w3mpro']
y = matches['w1mpro'] - matches['w2mpro']
m = 0.83809562126163
b = -1.045969175938493
def distance (m,b, x,y): 
   return np.abs(m * x + b-y)/np.sqrt(m**2 + 1**2)
matches['dist'] = distance(m,b, x,y)

In [17]:
matches['w2-w3'] = x 
matches['w1-w2'] = y
matches['w2-w3 ERR'] = np.sqrt(matches['w2sigmpro']**2 + matches['w3sigmpro']**2)
matches['w1-w2 ERR'] = np.sqrt(matches['w1sigmpro']**2 + matches['w2sigmpro']**2)


In [19]:
#loading in THC catalog; change path as needed
hdu_list = fits.open('./catalogs/THC_catalog.fits')
thc = Table(hdu_list[1].data)
hdu_list.close()

In [20]:
matches['THC'] = Column([''] * len(matches), dtype = 'U9')
for i in range(len(matches)): 
    if(matches['name'][i] in thc['WISEA']): 
        idx = np.where(thc['WISEA'] == matches['name'][i])[0][0]
        matches['THC'][i] = thc['THC'][idx]

In [23]:
matches['1eRASS'] = [name[7:23] for name in matches['1eRASS']]

In [25]:
#loading in TeVCAT (https://tevcat.org/) HBL catalog; change path as needed
tevcat= ascii.read('./catalogs/other_blazar-hbl_cats/HBL_newlist.csv', format='csv')

In [26]:
c = SkyCoord(ra = matches['ra']*u.degree, dec = matches['dec']*u.degree, frame = 'icrs')
catalog = SkyCoord(ra = tevcat['Simbad RA deg']*u.degree, dec = tevcat['Simbad DEC deg']*u.degree, frame = 'icrs')
idx, d2d, d3d = match_coordinates_sky(c, catalog)
m = d2d.arcsec <= 5
matches['TeVCAT Assoc'] = Column([''] * len(matches), dtype = 'U21')
matches['TeVCAT Assoc'][m] = tevcat['Name'][idx[m]]


In [28]:
final = Table([matches['THC'], matches['name'], matches['1eRASS'], matches['ra'], matches['dec'], matches['EXT_LIKE'], matches['w2-w3'], matches['w2-w3 ERR'], matches['w1-w2'], matches['w1-w2 ERR'], matches['dist'], matches['Alpha IRX'], matches['TeVCAT Assoc']])

In [29]:
final

THC,name,1eRASS,ra,dec,EXT_LIKE,w2-w3,w2-w3 ERR,w1-w2,w1-w2 ERR,dist,Alpha IRX,TeVCAT Assoc
str9,str19,str16,float64,float64,float32,float64,float64,float64,float64,float64,float64,str21
,J051639.13-844008.0,J051642.1-844007,79.1630516,-84.6688941,0.0,2.349,0.08509406559801923,0.3439999999999994,0.03894868418830089,0.44354247861439267,1.1527534338858887,
,J185159.49-841628.2,J185158.8-841625,282.9978981,-84.2745041,0.0,2.302999999999999,0.1275774274705365,0.5470000000000006,0.04036087214122113,0.25841111316630005,1.248067313632769,
,J075655.94-815653.4,J075655.2-815654,119.2330998,-81.948188,0.0,1.4100000000000001,0.11552056094046635,0.20599999999999952,0.03394112549695428,0.05384456462967376,1.4068596786153855,
,J091009.53-794627.6,J091008.7-794628,137.539722,-79.7743393,0.0,1.6300000000000008,0.032015621187164243,0.14700000000000024,0.029,0.13268831150623522,1.9158074073117017,
,J043124.15-793224.6,J043122.6-793225,67.8506267,-79.5401767,0.0,2.3889999999999993,0.041340053217188776,0.7829999999999995,0.0318276609256791,0.13277612562430968,1.2668721356997477,
,J130722.72-773722.9,J130723.2-773724,196.8446743,-77.6230362,0.0,1.6749999999999998,0.025612496949731396,0.5739999999999998,0.03047950130825634,0.16566928323212626,2.034770985471037,
,J082029.61-800325.6,J082029.0-800327,125.1233979,-80.0571241,0.0,1.8989999999999991,0.02765863337187866,0.3930000000000007,0.03114482300479487,0.1169365686566224,1.7524235326117097,
,J011947.05-790356.1,J011946.5-790354,19.9460557,-79.0655836,0.0,2.2990000000000013,0.08902246907382427,0.7479999999999993,0.03324154027718933,0.10179070357924803,1.212621320694974,
,J133051.33-770928.2,J133051.3-770930,202.7138944,-77.1578497,0.0,1.9540000000000006,0.11035397591387452,0.20400000000000063,0.0367967389859482,0.2971190325717619,1.3685424826353647,


In [30]:
#gamma-ray catalog matching; again, change the paths to match your local directory structure

#2FHL (Ackerman et al. 2016)
hdu_list = fits.open('./catalogs/gammaray/2FHL.fits')
fhl2 = Table(hdu_list[1].data)
c = SkyCoord(ra = final['ra']*u.degree, dec = final['dec']*u.degree, frame = 'fk5')
catalog = SkyCoord(ra = fhl2['RAJ2000']*u.degree, dec = fhl2['DEJ2000']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.deg <= fhl2['Pos_err_95'][idx]
final['2FHL Assoc'] = Column(['']*len(final), dtype = 'U17')
final['2FHL Assoc'][mask] = fhl2['Source_Name'][idx[mask]]
#3FHL (Ajello et al. 2017)
hdu_list = fits.open('./catalogs/gammaray/3FHL.fits')
fhl3 = Table(hdu_list[1].data)
catalog = SkyCoord(ra = fhl3['RAJ2000']*u.degree, dec = fhl3['DEJ2000']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.deg <= fhl3['Conf_95_SemiMajor'][idx]
final['3FHL Assoc'] = Column(['']*len(final), dtype = 'U18')
final['3FHL Assoc'][mask] = fhl3['Source_Name'][idx[mask]]
#4FGL_DR4 (Ballet et al. 2024)
hdu_list = fits.open('./catalogs/gammaray/4FGL_DR4.fits')
fgl4 = Table(hdu_list[1].data)
catalog = SkyCoord(ra = fgl4['RAJ2000']*u.degree, dec = fgl4['DEJ2000']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.deg <= fgl4['Conf_95_SemiMajor'][idx]
final['4FGL Assoc'] = Column(['']*len(final), dtype = 'U18')
final['4FGL Assoc'][mask] = fgl4['Source_Name'][idx[mask]]
final['4FGL Assoc'] = [name.replace(' ', '') for name in final['4FGL Assoc']]
final['4FGL Assoc'] = [name.replace('LJ', 'L J') for name in final['4FGL Assoc']]
#4LAC_DR3 (Ajello et al. 2023)
hdu_list = fits.open('./catalogs/gammaray/4LAC-DR3.fits')
lac = Table(hdu_list[1].data)
catalog = SkyCoord(ra = lac['RACdeg']*u.degree, dec = lac['DECdeg']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5.0
final['4LAC Assoc'] = Column(['']*len(final), dtype = 'U28')
final['4LAC Assoc'][mask] = lac['Assoc1'][idx[mask]]
#1CGH (Arsioli et al. 2025)
hdu_list = fits.open('./catalogs/gammaray/1CGH_Preliminary_V2.0.fits')
cgh = Table(hdu_list[1].data)
catalog = SkyCoord(ra = cgh['RA_1CGH']*u.degree, dec = cgh['DEC_1CGH']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5.0
final['1CGH Assoc'] = Column(['']*len(final), dtype = 'U28')
final['1CGH Assoc'][mask] = cgh['Counterpart_name'][idx[mask]]


/var/folders/2z/y_y_51l51r1cdd9b0xsl_qj80000gn/T/ipykernel_76516/2361240760.py:45: StringTruncateWarning: truncated right side string(s) longer than 28 character(s) during assignment
  final['1CGH Assoc'][mask] = cgh['Counterpart_name'][idx[mask]]


In [31]:
#multi-frequency catalog matching; again, change the paths to match your local directory structure

#BZCAT (Massaro et al. 2015)
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/bzcat.fits')
bzcat = Table(hdu_list[1].data)
catalog = SkyCoord(ra = bzcat['RA']*u.degree, dec = bzcat['DEC']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5
final['BZCAT Assoc'] = Column(['']*len(final), dtype = 'U15')
final['BZCAT OTYPE'] = Column(['']*len(final), dtype = 'U24')
for i in range(len(final)): 
    if(mask[i]): 
        name = bzcat['NAME'][idx[i]]
        name = name[10:]
        final['BZCAT Assoc'][i] = name
        final['BZCAT OTYPE'][i] = bzcat['OBJECT_TYPE'][idx[i]]
        final['BZCAT OTYPE'][i] = final['BZCAT OTYPE'][i].replace('  ', '')
#3HSP (Chang et al. 2019)
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/3HSP.fits')
hsp3 = Table(hdu_list[1].data)
catalog = SkyCoord(ra = hsp3['RAJ2000']*u.degree, dec = hsp3['DEJ2000']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5
final['3HSP Assoc'] = Column(['']*len(final), dtype = 'U20')
final['3HSP Assoc'][mask] = hsp3['Name'][idx[mask]]
#1WHSP (Arsioli et al. 2015)
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/1whsp.fits')
whsp1 = Table(hdu_list[1].data)
catalog = SkyCoord(ra = whsp1['_RA']*u.degree, dec = whsp1['_DE']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5
final['1WHSP Assoc'] = Column(['']*len(final), dtype = 'U15')
final['1WHSP Assoc'][mask] = whsp1['_1WHSP'][idx[mask]]
#2WHSP (Chang et al. 2017)
hdu_list= fits.open('./catalogs/other_blazar-hbl_cats/whsp2.fits')
whsp2 = Table(hdu_list[1].data)
catalog = SkyCoord(ra = whsp2['_RA']*u.degree, dec = whsp2['_DE']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5
final['2WHSP Assoc'] = Column(['']*len(final), dtype = 'U15')
final['2WHSP Assoc'][mask] = whsp2['_2WHSPJ'][idx[mask]]

In [32]:
#TeV-peaked candidate BL Lac objects (Costamante 2020)
C20 = ascii.read('./catalogs/other_blazar-hbl_cats/C20.txt')
C20 = C20[0][:]
final['C20'] = Column(['N']*len(final), dtype = 'U1')
for i in range(len(final)): 
    target = final['BZCAT Assoc'][i] 
    target = target.replace(" ", "")
    if(target != "None"): 
        target1 = target[0:9]
        for j in range(len(C20)): 
            c_t = C20[j]
            c_t1 = c_t[0:9]
            if(target1 == c_t1): 
                target2 = target[10:14]
                c_t2 = c_t[10:14]
                if(target2 == c_t2): 
                    final['C20'][i] = 'Y'
#Two new catalogs of blazar candidates in the WISE Infrared sky (D'Abrusco et al. 2019)
#T3
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/dabruscot3.fits')
dabruscot3 = Table(hdu_list[1].data)
#T5
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/dabruscot5.fits')
dabruscot5 = Table(hdu_list[1].data)
final['D19'] = Column(['N']*len(final), dtype = 'U1')
for i in range(len(final)): 
    target = final['name'][i]
    target1 = target[0:10]
    for j in range(len(dabruscot3)): 
        catalog = dabruscot3['WISE'][j]
        catalog1 = catalog[0:10]
        if(target1 == catalog1): 
           target2 = target[11:21]
           catalog2 = catalog[11:21]
           if(target2 == catalog2): 
               final['D19'][i] = 'Y'
    for k in range(len(dabruscot5)): 
        catalog = dabruscot5['WISE'][k]
        catalog1 = catalog[0:10]
        if(target1 == catalog1): 
            target2 = target[11:21]
            catalog2 = catalog[11:21]
            if(target2 == catalog2): 
                final['D19'][i] = 'Y'

In [33]:
#Exploring the Most Extreme Gamma-Ray Blazars Using Broadband Spectral Energy Distributions (Láinez et al. 2025)
L25 = ascii.read('./catalogs/other_blazar-hbl_cats/L25.txt')
L25 = L25[0][:]
final['L25'] = Column(['N']*len(final), dtype = 'U1')
for i in range(len(final)): 
    target = final['4FGL Assoc'][i] 
    target1 = target[5:12]
    for j in range(len(L25)): 
        l_t = L25[j]
        l_t1 = l_t[0:7]
        if(target1 == l_t1): 
            target2 =target[13:17]
            l_t2 = l_t[8:12]
            if(target2 == l_t2): 
                final['L25'][i] = 'Y'
#A new look at the extragalactic very high energy sky: Searching for TeV-emitting candidates among the X-ray-bright, non-Fermi-detected blazar population (Marchesi et al. 2025)                               
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/Marchesi_Xray_Bl_AA_25.fits')
marchesi = Table(hdu_list[1].data)
final['MAR25'] = Column(['N']*len(final), dtype = 'U1')
for i in range(len(final)): 
    target = final['BZCAT Assoc'][i]
    if(target != "None"):
        target = target.replace(" ", "")
        target1 = target[0:9]
        for j in range(len(marchesi)): 
            catalog = marchesi['ID_5BZCAT'][j]
            catalog1 = catalog[0:9]
            if(target1 == catalog1): 
                target2 = target[10:14]
                catalog2 = catalog[10:14]
                if(target2 == catalog2): 
                    final['MAR25'][i] = 'Y'
#BL Lac candidates for TeV observations (Massaro et al. 2013)
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/massaro_selection.fits')
massaro = Table(hdu_list[1].data)
final['MAS13'] = Column(['N']*len(final), dtype = 'U1')
for i in range(len(final)): 
    name = final['name'][i]
    if(name in massaro['WISE']): 
        final['MAS13'][i] = 'Y'
        idx = np.where(massaro['WISE'] == name)[0][0]


In [34]:
final.sort('ra')

In [35]:
final.rename_column('name', 'WISEA')

In [36]:
final.rename_column('ra', 'RA')
final.rename_column('dec', 'DEC')

In [40]:
tbl= QTable(final)
tbl['ra'] = tbl['RA']*u.degree 
tbl['dec'] = tbl['DEC']*u.degree
tbl['w2-w3'] = tbl['w2-w3']*u.mag
tbl['w2-w3 ERR'] = tbl['w2-w3 ERR']*u.mag
tbl['w1-w2'] = tbl['w1-w2']*u.mag
tbl['w1-w2 ERR'] = tbl['w1-w2 ERR']*u.mag
tbl['dist'] = tbl['dist']*u.mag

table_hdu = fits.BinTableHDU(tbl) 
header = table_hdu.header
header['EXTNAME'] = 'ALL_eROSITA_WISE_MATCHES'
header['AUTHOR'] = 'Cassie Metzger' 
header['DATE'] = '2025-04-16' 
header['WAVELENG'] = 'Multi'
header['DATASRC'] = 'eROSITA, WISE'
header['DOI'] = '10.48550/arXiv.2501.12520' 
header['WAVELENG'] = 'Multi'
header['SELPRO'] = 'All sources in the eROSITA catalog with WISE counterparts within 5'' that have IR magnitudes less than 14.3, 13.8, 12.2 in the W1, W2, W3 bands respectively and IR colors than fall between 1.25-2.4 and 0.05-0.9 in the W2-W3 and W1-W2 bands,respectively.'
tbl.write('./catalogs/WISE_EROSITA_ALLmatches.fits', format = 'fits', overwrite = True)
print('Catalog saved as WISE_EROSITA_ALLmatches.fits')

Catalog saved as WISE_EROSITA_ALLmatches.fits
